In [4]:
# Import libraries
import numpy as np

In [5]:
# Define sensitivity / elasticity function
def sensitivity_elasticity(A):
    eigvals, eigvecs = np.linalg.eig(A)
    idx = np.argmax(eigvals.real)
    lam = eigvals[idx].real
    w = eigvecs[:, idx].real

    # left eigenvector (reproductive value)
    eigvals_left, eigvecs_left = np.linalg.eig(A.T)
    idx_left = np.argmax(eigvals_left.real)
    v = eigvecs_left[:, idx_left].real

    # scale for v^T w = 1
    scalar = v @ w
    v = v / scalar

    # sensitivity: outer product of v and w
    S = np.outer(v, w)

    # elasticity (a_ij / lambda) * sensitivity
    E = (A / lam) * S

    return lam, w, v, S, E 

# Define function to compile 6-stage matrix
def compile_matrix(f, p, g):
    m = np.zeros((6, 6)) # Construct empty 6 x 6 matrix

    m[0, :] = f # Fertility rates

    for i in range(6): # Survival rates
        m[i, i] = p[i]
    
    for i in range(5):
        m[i+1, i] = g[i]
    
    return m

In [10]:
# Historical matrix
f = np.array([0.000, 0.000, 0.299, 0.774, 1.957, 6.025]) # Fertility
p = np.array([0.903, 0.961, 0.965, 0.976, 0.963, 0.990]) # Survival
g = np.array([0.004, 0.012, 0.017, 0.012, 0.018, 0.000]) # Growth

h = compile_matrix(f, p, g)
lam, w, v, S, E = sensitivity_elasticity(h)

print('The dominant eigenvalue of h is:', lam)
print('')
print('The elasticity matrix of h is:')
for row in E:
    print(row)

total = 0
for row in E:
    for col in row:
        total += col
print('')
print('Sanity check on elasticity calculation (should equal 1): ', total)

The dominant eigenvalue of h is: 1.0017812237824297

The elasticity matrix of h is:
[0.05446765 0.         0.00057713 0.00098513 0.00077073 0.00362535]
[0.00595834 0.14040689 0.         0.         0.         0.        ]
[0.         0.00595834 0.15632427 0.         0.         0.        ]
[0.         0.         0.00538121 0.20371635 0.         0.        ]
[0.         0.         0.         0.00439608 0.1091617  0.        ]
[0.         0.         0.         0.         0.00362535 0.30464549]

Sanity check on elasticity calculation (should equal 1):  1.0
